<a href="https://colab.research.google.com/github/FeraMaks/test/blob/main/CNN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

##Device configuration

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##Hyper parameters

In [3]:
num_epochs = 12
batch_size = 4
lr = 0.001

dataset has PILImage images of range [0,1]
we transform them to Tensors of normalizer range [-1, 1]

In [4]:
transform = transforms.Compose(
                        [transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                           shuffle=False)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


##Implement CNN

In [5]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
model = ConvNet().to(device)

L = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=lr)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    #Forward pass
    outputs = model(images)
    loss = L(outputs, labels)

    #Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
  print('Finished Training')

Epoch [1/12], Step [2000/12500], Loss: 2.3048
Epoch [1/12], Step [4000/12500], Loss: 2.2969
Epoch [1/12], Step [6000/12500], Loss: 2.3004
Epoch [1/12], Step [8000/12500], Loss: 2.3021
Epoch [1/12], Step [10000/12500], Loss: 2.2979
Epoch [1/12], Step [12000/12500], Loss: 2.2765
Finished Training
Epoch [2/12], Step [2000/12500], Loss: 2.2170
Epoch [2/12], Step [4000/12500], Loss: 2.2170
Epoch [2/12], Step [6000/12500], Loss: 1.8947
Epoch [2/12], Step [8000/12500], Loss: 1.8913
Epoch [2/12], Step [10000/12500], Loss: 1.8236
Epoch [2/12], Step [12000/12500], Loss: 2.1252
Finished Training
Epoch [3/12], Step [2000/12500], Loss: 1.7358
Epoch [3/12], Step [4000/12500], Loss: 1.6769
Epoch [3/12], Step [6000/12500], Loss: 2.9624
Epoch [3/12], Step [8000/12500], Loss: 1.2498
Epoch [3/12], Step [10000/12500], Loss: 3.2148
Epoch [3/12], Step [12000/12500], Loss: 1.4218
Finished Training
Epoch [4/12], Step [2000/12500], Loss: 1.4034
Epoch [4/12], Step [4000/12500], Loss: 1.2631
Epoch [4/12], Step [

In [6]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    #max returns (value, index)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1
  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc} %')

  for i in range(10):
    acc = 100.0 *n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 59.36 %
Accuracy of plane: 59.1 %
Accuracy of car: 72.7 %
Accuracy of bird: 32.9 %
Accuracy of cat: 54.3 %
Accuracy of deer: 47.7 %
Accuracy of dog: 41.3 %
Accuracy of frog: 74.3 %
Accuracy of horse: 63.4 %
Accuracy of ship: 82.7 %
Accuracy of truck: 65.2 %
